In [5]:
# import necessary libraries
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
power = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip", sep = ';', header=0, low_memory=False,na_values = '?', parse_dates={"Datetime":['Date','Time']},index_col=["Datetime"])
power_data = power.copy()
power_data.head()

In [ ]:
power_data.replace({'?': np.nan},inplace=True)
power_data.isnull().sum()

In [ ]:
for column in power_data.select_dtypes(include=['object']).columns:
  if column != 'Time':
    power_data[[column]] = power_data[[column]].astype('float')

power_data.info()

In [ ]:
power_data = power_data.fillna(power_data.mean())
power_data.isnull().sum()

In [ ]:
power_daily = power_data.resample('D').sum()
power_daily.info()

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(power_daily.index, power_daily.Global_active_power, '--')
plt.grid()
plt.xlabel('Day')
plt.ylabel('G_A_P')

In [ ]:
print(round(power_daily['Global_active_power'].corr(power_daily['Global_reactive_power']), 2))

In [ ]:
print(round(power_daily['Global_intensity'].corr(power_daily['Voltage']), 2))

In [ ]:
power_daily = power_daily.reset_index()
power_daily.head()

In [ ]:
model_data = power_daily[['Datetime', 'Global_active_power']]
model_data = model_data.rename(columns={'Datetime':'ds', 'Global_active_power':'y'})

model_data.head()

In [ ]:
train = model_data[:-365]
test = model_data[-365:]
print('train shape', train.shape)
print('test shape', test.shape)

In [ ]:
from fbprophet import Prophet
model = Prophet()
model.fit(train)

In [ ]:
forecast = model.predict(test) 
forecast.head()

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']]

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(np.array(test['y']),np.array(forecast['yhat'])))
mape = np.mean(np.abs(np.array(test['y']) - np.array(forecast.yhat[-365:]))/ np.array(test['y'])) *100
 
print('mean_absolute_percentage_error', round(mape,2))
print('root mean_squared_error', round(rmse,2))

In [ ]:
from fbprophet.plot import plot_yearly
plot_yearly(model)

In [ ]:
 multi_model = power_daily.rename(columns = {'Datetime':'ds','Global_active_power':'y',
                                           'Global_reactive_power':'add1','Voltage':'add2',
                                           'Global_intensity':'add3','Sub_metering_1':'add4',
                                          'Sub_metering_2':'add5','Sub_metering_3':'add6'})
multi_model.head()

In [ ]:
train2 = multi_model[:-365]
test2 = multi_model[-365:]
print('train shape', train2.shape)
print('test shape', test2.shape)

In [ ]:
model2 = Prophet()
model2.add_regressor('add1')
model2.add_regressor('add2')
model2.add_regressor('add3')
model2.add_regressor('add4')
model2.add_regressor('add5')
model2.add_regressor('add6')

In [ ]:
 
model2.fit(train2)
 
forecast2 = model2.predict(test2)

In [ ]:
forecast2

In [ ]:
mape2 = np.mean(np.abs(np.array(test2['y']) -np.array(forecast2['yhat']))/ np.array(test2['y'])) *100
round(mape2,2)

In [ ]:
rmse2 = np.sqrt(mean_squared_error(np.array(test2['y']),np.array(forecast2['yhat'])))
round(rmse2,2)

In [ ]:
from fbprophet.plot import plot_weekly
plot_weekly(model2)